<a href="https://colab.research.google.com/github/Kimurist2024/Kimurist/blob/main/%E3%83%A2%E3%83%87%E3%83%AB%E3%83%9E%E3%83%BC%E3%82%B8%E3%80%80chie%2BWiki026.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/arcee-ai/mergekit.git
!pip install -e ./mergekit

Cloning into 'mergekit'...
remote: Enumerating objects: 2685, done.
remote: Counting objects: 100% (994/994), done.
remote: Compressing objects: 100% (277/277), done.
remote: Total 2685 (delta 865), reused 717 (delta 717), pack-reused 1691 (from 4)
Receiving objects: 100% (2685/2685), 922.36 KiB | 12.14 MiB/s, done.
Resolving deltas: 100% (1825/1825), done.
Obtaining file:///content/mergekit
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.8 MB/s eta 0:00:00

In [ ]:
!mkdir -p examples

In [ ]:
%%writefile examples/llm-jp-linear.yaml
models:
  - model: "model1.safetensors"
    parameters:
      weight: 0.7
  - model: "model2.safetensors"
    parameters:
      weight: 0.3
merge_method: linear
dtype: float16

Writing examples/llm-jp-linear.yaml


In [ ]:
!pip install --upgrade safetensors


In [ ]:
import json
import os

def create_tokenizer_config(path="/content/sample_data/tokenizer_config.json"):
    # トークナイザーの設定
    config = {
        "tokenizer_class": "AutoTokenizer",
        "model_max_length": 512,
        "padding_side": "right",
        "truncation_side": "right",
        "special_tokens_map": {
            "bos_token": "<s>",
            "eos_token": "</s>",
            "unk_token": "<unk>",
            "pad_token": "<pad>",
            "mask_token": "<mask>"
        }
    }

    # ディレクトリ作成（必要な場合）
    os.makedirs(os.path.dirname(path), exist_ok=True)

    # JSONファイルを書き出し
    with open(path, "w") as f:
        json.dump(config, f, indent=4, ensure_ascii=False)

    print(f"{path} を作成しました")

# 実行
create_tokenizer_config()


/content/sample_data/tokenizer_config.json を作成しました


In [ ]:
import subprocess
import os

def create_merge_config():
    """merge_config.ymlを作成する"""
    config_content = """
merge_method: linear
models:
  - model:
      path: "/content/sample_data/model1.safetensors"
    alpha: 0.6
  - model:
      path: "/content/sample_data/model2.safetensors"
    alpha: 0.4
tokenizer_source: "/content/sample_data"
dtype: float16
"""
    with open('merge_config.yml', 'w') as f:
        f.write(config_content)
    print(" merge_config.yml を作成しました")

def run_mergekit():
    """MergeKitでモデルをマージする"""
    config_path = "merge_config.yml"
    output_dir = "./merged_model"

    command = [
        "mergekit-yaml", config_path, output_dir, "--cuda", "--lazy-unpickle", "--allow-crimes"
    ]



In [ ]:
from safetensors.torch import load_file

def inspect_safetensors(model_path):
    """safetensorsファイル内のテンソル情報を表示"""
    try:
        tensors = load_file(model_path)
        print(f"\n モデル: {model_path}")
        print("="*50)
        for key, tensor in tensors.items():
            print(f"🔹 {key}: shape={tensor.shape}, dtype={tensor.dtype}")
        print("="*50)
    except Exception as e:
        print(f" 読み込みエラー: {model_path}\n{e}")

# 実行
inspect_safetensors("model1.safetensors")
inspect_safetensors("model2.safetensors")


 読み込みエラー: model1.safetensors
Error while deserializing header: MetadataIncompleteBuffer

 モデル: model2.safetensors
🔹 decoder.block.0.layer.0.SelfAttention.k.weight: shape=torch.Size([512, 512]), dtype=torch.float32
🔹 decoder.block.0.layer.0.SelfAttention.o.weight: shape=torch.Size([512, 512]), dtype=torch.float32
🔹 decoder.block.0.layer.0.SelfAttention.q.weight: shape=torch.Size([512, 512]), dtype=torch.float32
🔹 decoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight: shape=torch.Size([32, 8]), dtype=torch.float32
🔹 decoder.block.0.layer.0.SelfAttention.v.weight: shape=torch.Size([512, 512]), dtype=torch.float32
🔹 decoder.block.0.layer.0.layer_norm.weight: shape=torch.Size([512]), dtype=torch.float32
🔹 decoder.block.0.layer.1.EncDecAttention.k.weight: shape=torch.Size([512, 512]), dtype=torch.float32
🔹 decoder.block.0.layer.1.EncDecAttention.o.weight: shape=torch.Size([512, 512]), dtype=torch.float32
🔹 decoder.block.0.layer.1.EncDecAttention.q.weight: shape=torch.Size([512

In [ ]:
from safetensors.torch import load_file

def inspect_safetensors(model_path):
    """safetensorsファイル内のテンソル情報を表示"""
    try:
        tensors = load_file(model_path)
        print(f"\n モデル: {model_path}")
        print("="*50)
        for key, tensor in tensors.items():
            print(f"🔹 {key}: shape={tensor.shape}, dtype={tensor.dtype}")
        print("="*50)
    except Exception as e:
        print(f" 読み込みエラー: {model_path}\n{e}")

# 実行
inspect_safetensors("model1.safetensors")




 モデル: model1.safetensors
🔹 decoder.block.0.layer.0.SelfAttention.k.weight: shape=torch.Size([768, 768]), dtype=torch.float32
🔹 decoder.block.0.layer.0.SelfAttention.o.weight: shape=torch.Size([768, 768]), dtype=torch.float32
🔹 decoder.block.0.layer.0.SelfAttention.q.weight: shape=torch.Size([768, 768]), dtype=torch.float32
🔹 decoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight: shape=torch.Size([32, 12]), dtype=torch.float32
🔹 decoder.block.0.layer.0.SelfAttention.v.weight: shape=torch.Size([768, 768]), dtype=torch.float32
🔹 decoder.block.0.layer.0.layer_norm.weight: shape=torch.Size([768]), dtype=torch.float32
🔹 decoder.block.0.layer.1.EncDecAttention.k.weight: shape=torch.Size([768, 768]), dtype=torch.float32
🔹 decoder.block.0.layer.1.EncDecAttention.o.weight: shape=torch.Size([768, 768]), dtype=torch.float32
🔹 decoder.block.0.layer.1.EncDecAttention.q.weight: shape=torch.Size([768, 768]), dtype=torch.float32
🔹 decoder.block.0.layer.1.EncDecAttention.v.weight: shape=

In [ ]:
from safetensors.torch import load_file

def inspect_safetensors(model_path):
    """safetensorsファイル内のテンソル情報を表示"""
    try:
        tensors = load_file(model_path)
        print(f"\n モデル: {model_path}")
        print("="*50)
        for key, tensor in tensors.items():
            print(f"🔹 {key}: shape={tensor.shape}, dtype={tensor.dtype}")
        print("="*50)
    except Exception as e:
        print(f" 読み込みエラー: {model_path}\n{e}")

# 実行
inspect_safetensors("model2.safetensors")



 モデル: model2.safetensors
🔹 decoder.block.0.layer.0.SelfAttention.k.weight: shape=torch.Size([512, 512]), dtype=torch.float32
🔹 decoder.block.0.layer.0.SelfAttention.o.weight: shape=torch.Size([512, 512]), dtype=torch.float32
🔹 decoder.block.0.layer.0.SelfAttention.q.weight: shape=torch.Size([512, 512]), dtype=torch.float32
🔹 decoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight: shape=torch.Size([32, 8]), dtype=torch.float32
🔹 decoder.block.0.layer.0.SelfAttention.v.weight: shape=torch.Size([512, 512]), dtype=torch.float32
🔹 decoder.block.0.layer.0.layer_norm.weight: shape=torch.Size([512]), dtype=torch.float32
🔹 decoder.block.0.layer.1.EncDecAttention.k.weight: shape=torch.Size([512, 512]), dtype=torch.float32
🔹 decoder.block.0.layer.1.EncDecAttention.o.weight: shape=torch.Size([512, 512]), dtype=torch.float32
🔹 decoder.block.0.layer.1.EncDecAttention.q.weight: shape=torch.Size([512, 512]), dtype=torch.float32
🔹 decoder.block.0.layer.1.EncDecAttention.v.weight: shape=t

In [ ]:
def create_merge_config(model1_path, model2_path, output_yaml="merge_config.yml", w1=0.6, w2=0.4):
    config = f"""
merge_method: linear
models:
  - model:
      path: "{model1_path}"
    alpha: {w1}
  - model:
      path: "{model2_path}"
    alpha: {w2}
tokenizer_source: "/content/sample_data"
dtype: float16
"""
    with open(output_yaml, "w") as f:
        f.write(config)
    print(f"✅ merge_config.yml を生成しました: {output_yaml}")

# 実行
create_merge_config("model1.safetensors", "model2.safetensors")


✅ merge_config.yml を生成しました: merge_config.yml


In [ ]:
import torch
from safetensors.torch import load_file, save_file
import subprocess

def merge_safetensors(model1_path, model2_path, output_path="merged_model.safetensors", w1=0.6, w2=0.4):
    # モデルを読み込む
    m1, m2 = load_file(model1_path), load_file(model2_path)
    merged = {}

    # model1 のテンソルを基準に model2 を調整
    for k, v1 in m1.items():
        t2 = m2.get(k, torch.zeros_like(v1))  # model2 にキーがない場合はゼロ埋め
        t2 = t2.to(v1.dtype)  # データ型を統一

        # 形状調整
        if v1.shape != t2.shape:
            print(f"🔧 {k}: {t2.shape} → {v1.shape} に調整中")

            if len(v1.shape) == 1:
                t2 = torch.nn.functional.pad(t2, (0, max(0, v1.shape[0] - t2.shape[0])))[:v1.shape[0]]

            elif len(v1.shape) == 2 and t2.numel() == v1.numel():
                t2 = t2.reshape(v1.shape)

            elif len(v1.shape) > 2 and t2.numel() == v1.numel():
                t2 = t2.reshape(v1.shape)

            else:
                print(f"⚠️ {k}: 形状不一致のためランダム初期化")
                t2 = torch.randn(v1.shape, dtype=v1.dtype)

        # 指定した比率でモデルをマージ
        merged[k] = w1 * v1 + w2 * t2

    # マージ結果を保存
    save_file(merged, output_path)
    print(f"✅ モデルマージ完了: {output_path}")

    # `merge_config.yml` を生成
    config = f"""
merge_method: linear
models:
  - model:
      path: "{model1_path}"
    alpha: {w1}
  - model:
      path: "{model2_path}"
    alpha: {w2}
tokenizer_source: "/content/sample_data"
dtype: float16
"""
    with open("merge_config.yml", "w") as f:
        f.write(config)
    print("✅ merge_config.yml を生成しました")

    # `mergekit` でマージを実行
    subprocess.run(["mergekit-yaml", "merge_config.yml", output_path, "--cuda", "--lazy-unpickle", "--allow-crimes"])
    print(f"✅ mergekit による最終マージ完了: {output_path}")

# 実行
merge_safetensors("model1.safetensors", "model2.safetensors")


🔧 decoder.block.0.layer.0.SelfAttention.k.weight: torch.Size([512, 512]) → torch.Size([768, 768]) に調整中
⚠️ decoder.block.0.layer.0.SelfAttention.k.weight: 形状不一致のためランダム初期化
🔧 decoder.block.0.layer.0.SelfAttention.o.weight: torch.Size([512, 512]) → torch.Size([768, 768]) に調整中
⚠️ decoder.block.0.layer.0.SelfAttention.o.weight: 形状不一致のためランダム初期化
🔧 decoder.block.0.layer.0.SelfAttention.q.weight: torch.Size([512, 512]) → torch.Size([768, 768]) に調整中
⚠️ decoder.block.0.layer.0.SelfAttention.q.weight: 形状不一致のためランダム初期化
🔧 decoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight: torch.Size([32, 8]) → torch.Size([32, 12]) に調整中
⚠️ decoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight: 形状不一致のためランダム初期化
🔧 decoder.block.0.layer.0.SelfAttention.v.weight: torch.Size([512, 512]) → torch.Size([768, 768]) に調整中
⚠️ decoder.block.0.layer.0.SelfAttention.v.weight: 形状不一致のためランダム初期化
🔧 decoder.block.0.layer.0.layer_norm.weight: torch.Size([512]) → torch.Size([768]) に調整中
🔧 decoder.block.0.layer

In [ ]:
import torch
from safetensors.torch import load_file, save_file
import subprocess
import json
import os

def merge_safetensors(model1_path, model2_path, output_path="merged_model.safetensors", w1=0.6, w2=0.4):
    # モデルを読み込む
    m1, m2 = load_file(model1_path), load_file(model2_path)
    merged = {}

    # model1 のテンソルを基準に model2 を調整
    for k, v1 in m1.items():
        t2 = m2.get(k, torch.zeros_like(v1))
        t2 = t2.to(v1.dtype)

        if v1.shape != t2.shape:
            print(f"🔧 {k}: {t2.shape} → {v1.shape} に調整中")

            if len(v1.shape) == 1:
                t2 = torch.nn.functional.pad(t2, (0, max(0, v1.shape[0] - t2.shape[0])))[:v1.shape[0]]
            elif len(v1.shape) == 2 and t2.numel() == v1.numel():
                t2 = t2.reshape(v1.shape)
            elif len(v1.shape) > 2 and t2.numel() == v1.numel():
                t2 = t2.reshape(v1.shape)
            else:
                print(f"⚠️ {k}: 形状不一致のためランダム初期化")
                t2 = torch.randn(v1.shape, dtype=v1.dtype)

        merged[k] = w1 * v1 + w2 * t2

    # マージ結果を保存
    save_file(merged, output_path)
    print(f"✅ モデルマージ完了: {output_path}")

    # `merge_config.yml` を生成
    config_yml = f"""
merge_method: linear
models:
  - model:
      path: "{model1_path}"
    alpha: {w1}
  - model:
      path: "{model2_path}"
    alpha: {w2}
tokenizer_source: "/content/sample_data"
dtype: float16
"""
    with open("merge_config.yml", "w") as f:
        f.write(config_yml)
    print("✅ merge_config.yml を作成しました")

    # `mergekit` でマージを実行
    subprocess.run(["mergekit-yaml", "merge_config.yml", output_path, "--cuda", "--lazy-unpickle", "--allow-crimes"])
    print(f"✅ mergekit による最終マージ完了: {output_path}")

    # **✅ `config.json` を生成**
    generate_config(output_path)

def generate_config(model_path, config_path="merged_model/config.json"):
    tensors = load_file(model_path)

    # ディレクトリが存在しない場合は作成
    config_dir = os.path.dirname(config_path)
    if config_dir and not os.path.exists(config_dir):
        os.makedirs(config_dir)

    # 隠れ層サイズ (hidden_size)
    hidden_size = tensors["decoder.block.0.layer.0.SelfAttention.k.weight"].shape[0]

    # レイヤー数 (num_hidden_layers) をカウント
    num_hidden_layers = max(
        int(k.split(".")[2]) for k in tensors.keys() if "decoder.block" in k
    ) + 1  # 最大の X + 1

    # アテンションヘッド数 (num_attention_heads)
    num_attention_heads = tensors["decoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight"].shape[1]

    # FFN の中間層サイズ (intermediate_size)
    intermediate_size = tensors["decoder.block.0.layer.2.DenseReluDense.wi_0.weight"].shape[0]

    # 語彙サイズ (vocab_size)
    vocab_size = tensors["shared.weight"].shape[0]

    # `config.json` の構造
    config = {
        "model_type": "transformer",
        "hidden_size": hidden_size,
        "num_attention_heads": num_attention_heads,
        "num_hidden_layers": num_hidden_layers,
        "intermediate_size": intermediate_size,
        "vocab_size": vocab_size,
        "max_position_embeddings": 1024,  # 一般的な値（要確認）
        "dtype": str(tensors["decoder.block.0.layer.0.SelfAttention.k.weight"].dtype)
    }

    # JSONファイルとして保存
    with open(config_path, "w") as f:
        json.dump(config, f, indent=4)

    print(f"✅ `config.json` を作成しました: {config_path}")

# 実行
merge_safetensors("model1.safetensors", "model2.safetensors")


🔧 decoder.block.0.layer.0.SelfAttention.k.weight: torch.Size([512, 512]) → torch.Size([768, 768]) に調整中
⚠️ decoder.block.0.layer.0.SelfAttention.k.weight: 形状不一致のためランダム初期化
🔧 decoder.block.0.layer.0.SelfAttention.o.weight: torch.Size([512, 512]) → torch.Size([768, 768]) に調整中
⚠️ decoder.block.0.layer.0.SelfAttention.o.weight: 形状不一致のためランダム初期化
🔧 decoder.block.0.layer.0.SelfAttention.q.weight: torch.Size([512, 512]) → torch.Size([768, 768]) に調整中
⚠️ decoder.block.0.layer.0.SelfAttention.q.weight: 形状不一致のためランダム初期化
🔧 decoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight: torch.Size([32, 8]) → torch.Size([32, 12]) に調整中
⚠️ decoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight: 形状不一致のためランダム初期化
🔧 decoder.block.0.layer.0.SelfAttention.v.weight: torch.Size([512, 512]) → torch.Size([768, 768]) に調整中
⚠️ decoder.block.0.layer.0.SelfAttention.v.weight: 形状不一致のためランダム初期化
🔧 decoder.block.0.layer.0.layer_norm.weight: torch.Size([512]) → torch.Size([768]) に調整中
🔧 decoder.block.0.layer

In [ ]:
import os
print(os.path.exists("merged_model/config.json"))  # True なら成功

# `config.json` の内容を表示
with open("merged_model/config.json", "r") as f:
    print(json.load(f))


True
{'model_type': 'transformer', 'hidden_size': 768, 'num_attention_heads': 12, 'num_hidden_layers': 12, 'intermediate_size': 2048, 'vocab_size': 32128, 'max_position_embeddings': 1024, 'dtype': 'torch.float32'}


In [ ]:
from google.colab import files
files.download("merged_model/config.json")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from safetensors.torch import load_file

# safetensors の中身を確認
tensors = load_file("merged_model.safetensors")

# テンソルのキー一覧を表示
for key, tensor in tensors.items():
    print(f"{key}: shape={tensor.shape}, dtype={tensor.dtype}")


decoder.block.0.layer.0.SelfAttention.k.weight: shape=torch.Size([768, 768]), dtype=torch.float32
decoder.block.0.layer.0.SelfAttention.o.weight: shape=torch.Size([768, 768]), dtype=torch.float32
decoder.block.0.layer.0.SelfAttention.q.weight: shape=torch.Size([768, 768]), dtype=torch.float32
decoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight: shape=torch.Size([32, 12]), dtype=torch.float32
decoder.block.0.layer.0.SelfAttention.v.weight: shape=torch.Size([768, 768]), dtype=torch.float32
decoder.block.0.layer.0.layer_norm.weight: shape=torch.Size([768]), dtype=torch.float32
decoder.block.0.layer.1.EncDecAttention.k.weight: shape=torch.Size([768, 768]), dtype=torch.float32
decoder.block.0.layer.1.EncDecAttention.o.weight: shape=torch.Size([768, 768]), dtype=torch.float32
decoder.block.0.layer.1.EncDecAttention.q.weight: shape=torch.Size([768, 768]), dtype=torch.float32
decoder.block.0.layer.1.EncDecAttention.v.weight: shape=torch.Size([768, 768]), dtype=torch.float32
de

In [ ]:
import json
from safetensors.torch import load_file

def generate_config(model_path, config_path="merged_model/config.json"):
    tensors = load_file(model_path)

    # 隠れ層サイズ (hidden_size)
    hidden_size = tensors["decoder.block.0.layer.0.SelfAttention.k.weight"].shape[0]

    # レイヤー数 (num_hidden_layers) をカウント
    num_hidden_layers = max(
        int(k.split(".")[2]) for k in tensors.keys() if "decoder.block" in k
    ) + 1  # 最大の X + 1

    # アテンションヘッド数 (num_attention_heads)
    num_attention_heads = tensors["decoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight"].shape[1]

    # FFN の中間層サイズ (intermediate_size)
    intermediate_size = tensors["decoder.block.0.layer.2.DenseReluDense.wi_0.weight"].shape[0]

    # 語彙サイズ (vocab_size)
    vocab_size = tensors["shared.weight"].shape[0]

    # `config.json` の構造
    config = {
        "model_type": "transformer",
        "hidden_size": hidden_size,
        "num_attention_heads": num_attention_heads,
        "num_hidden_layers": num_hidden_layers,
        "intermediate_size": intermediate_size,
        "vocab_size": vocab_size,
        "max_position_embeddings": 1024,  # 一般的な値（要確認）
        "dtype": str(tensors["decoder.block.0.layer.0.SelfAttention.k.weight"].dtype)
    }

    # JSONファイルとして保存
    with open(config_path, "w") as f:
        json.dump(config, f, indent=4)

    print(f"✅ `config.json` を作成しました: {config_path}")

# 実行
generate_config("merged_model.safetensors")


FileNotFoundError: [Errno 2] No such file or directory: 'merged_model/config.json'

In [ ]:
!ls -l
# または
!ls -l merged_model


total 2170672
drwxr-xr-x 2 root root      4096 Feb 19 00:28 examples
-rw-r--r-- 1 root root       212 Feb 19 00:44 merge_config.yml
drwxr-xr-x 9 root root      4096 Feb 19 00:19 mergekit
-rw-r--r-- 1 root root 990345064 Feb 19 00:42 model1.safetensors
-rw-r--r-- 1 root root 990345032 Feb 19 00:44 model2_aligned.safetensors
-rw-r--r-- 1 root root 242041896 Feb 19 00:06 model2.safetensors
drwxr-xr-x 1 root root      4096 Feb 19 00:28 sample_data
ls: cannot access 'merged_model': No such file or directory


In [ ]:
!mergekit-yaml merge_config.yml /content/merged_model.safetensors --cuda --lazy-unpickle --allow-crimes


2025-02-19 01:30:16.283590: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739928616.305644   23021 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739928616.312112   23021 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-19 01:30:16.332790: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Traceback (most recent call last):
  File "/usr/local/bin/mergekit-yaml", line 8, in <module>
    sys.exit(main())
          

In [ ]:
from google.colab import files

# マージ後のモデルをダウンロード
files.download("/content/merged_model.safetensors")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!7z a "/content/merged_model.7z" "/content/merged_model.safetensors"


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,8 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive:
  0M Scan  /content/                    1 file, 990345032 bytes (945 MiB)

Creating archive: /content/merged_model.7z

Items to compress: 1

  0%      0% + merged_model.safetensors                                 1% + merged_model.safetensors                                 2% + merged_model.safetensors                                 3% + merged_model.safetensors                                 4% + merged_model.safetensors                           

In [ ]:

from google.colab import files
files.download("/content/merged_model.7z")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from safetensors.torch import load_file

# モデルをロード
model_path = "/content/merged_model.safetensors"
model = load_file(model_path)

# モデル内のテンソル情報を表示
print(f"📊 モデル '{model_path}' 内のテンソル情報:\n")
for k, v in model.items():
    print(f"🔹 {k}: shape={v.shape}, dtype={v.dtype}, mean={v.mean():.4f}, std={v.std():.4f}")

print(f"\n✅ {len(model)}個のテンソルがロードされました")


FileNotFoundError: No such file or directory: "/content/merged_model.safetensors"

In [ ]:
import torch
import time
from safetensors.torch import load_file

# モデルをロード
model_path = "/content/merged_model.safetensors"
model = load_file(model_path)

# テンソルごとの推論速度測定
start_time = time.time()
for i, (k, v) in enumerate(model.items()):
    _ = v * 2  # ダミー計算
    if i >= 100:
        break
end_time = time.time()

print(f" 100テンソルの計算時間: {end_time - start_time:.4f}秒")


 100テンソルの計算時間: 0.0208秒


In [ ]:
import torch
import time
from safetensors.torch import load_file

# モデルをロード
model_path = "/content/model2.safetensors"
model = load_file(model_path)

# テンソルごとの推論速度測定
start_time = time.time()
for i, (k, v) in enumerate(model.items()):
    _ = v * 2  # ダミー計算
    if i >= 100:
        break
end_time = time.time()

print(f" 100テンソルの計算時間: {end_time - start_time:.4f}秒")

 100テンソルの計算時間: 0.0134秒


In [ ]:
import torch
import time
from safetensors.torch import load_file

# モデルをロード
model_path = "/content/model2.safetensors"
model = load_file(model_path)

# テンソルごとの推論速度測定
start_time = time.time()
for i, (k, v) in enumerate(model.items()):
    _ = v * 2  # ダミー計算
    if i >= 100:
        break
end_time = time.time()

print(f" 100テンソルの計算時間: {end_time - start_time:.4f}秒")

 100テンソルの計算時間: 0.0113秒


In [ ]:
import json
import os

def create_tokenizer_config():
    config = {
        "tokenizer_class": "AutoTokenizer",
        "model_max_length": 512,
        "padding_side": "right",
        "truncation_side": "right",
        "special_tokens_map": {
            "bos_token": "<s>",
            "eos_token": "</s>",
            "unk_token": "<unk>",
            "pad_token": "<pad>",
            "mask_token": "<mask>"
        }
    }

    os.makedirs("/content/sample_data", exist_ok=True)
    with open("/content/sample_data/tokenizer_config.json", "w") as f:
        json.dump(config, f, indent=4)

    print(" tokenizer_config.json を作成しました")

create_tokenizer_config()


 tokenizer_config.json を作成しました


In [ ]:

!wget -P /content/sample_data https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt


--2025-02-17 10:13:18--  https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt
Resolving huggingface.co (huggingface.co)... 13.35.202.34, 13.35.202.121, 13.35.202.97, ...
Connecting to huggingface.co (huggingface.co)|13.35.202.34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 231508 (226K) [text/plain]
Saving to: ‘/content/sample_data/vocab.txt.1’

vocab.txt.1         100%[===================>] 226.08K  1.03MB/s    in 0.2s    

2025-02-17 10:13:18 (1.03 MB/s) - ‘/content/sample_data/vocab.txt.1’ saved [231508/231508]



In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import time

# モデル＆トークナイザーをロード（tokenizerをmodelパスから自動取得）
model_path = "/content/merged_model.safetensors"
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# Define torch_dtype
torch_dtype = torch.float16


# ... (rest of your code) ...

In [ ]:
pip install transformers torch datasets scikit-learn


In [ ]:
import json

config = {
    "architectures": ["BertForSequenceClassification"],
    "model_type": "bert",
    "num_labels": 2,
    "hidden_size": 768,
    "num_attention_heads": 12,
    "num_hidden_layers": 12,
    "vocab_size": 30522
}

with open("/content/sample_data/config.json", "w") as f:
    json.dump(config, f, indent=4)

print("✅ config.json を作成しました")


✅ config.json を作成しました


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
from safetensors.torch import load_file
import torch

# モデルのディレクトリパス
model_dir = "/content/merged_model.safetensors"

# トークナイザーのロード
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# モデルの構成情報を作成
num_labels = 2  # 例: バイナリ分類の場合
config = AutoConfig.from_pretrained("bert-base-uncased", num_labels=num_labels)
model = AutoModelForSequenceClassification.from_config(config, torch_dtype=torch.float16)

# safetensorsファイルをロードして、キーの修正
state_dict = load_file(model_dir)

# キー名修正（prefixの不一致を解消）
corrected_state_dict = {}
for key, value in state_dict.items():
    # 'encoder.'が必要な場合などに対応
    new_key = key.replace("encoder.block", "bert.encoder.layer")
    new_key = new_key.replace("embeddings", "bert.embeddings")
    new_key = new_key.replace("decoder", "bert")
    corrected_state_dict[new_key] = value

# モデルに修正済みのstate_dictをロード
model.load_state_dict(corrected_state_dict, strict=False)

# デバイスの設定（GPUが利用可能ならGPUを使用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(" モデルのロードが完了しました！")


 モデルのロードが完了しました！


In [ ]:
# 推論テスト
input_text = "The future of AI is"
inputs = tokenizer(input_text, return_tensors="pt").to(device)

# 推論実行
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=-1).item()

print(f" 推論結果: {predicted_class}")


 推論結果: 0


In [ ]:
# 推論テスト：出力内容の確認
input_text = "The future of AI is"
inputs = tokenizer(input_text, return_tensors="pt").to(device)

# 推論実行
model.eval()
with torch.no_grad():
    outputs = model(**inputs)

# 出力内容を確認
print(f"モデル出力の内容: {outputs}")

# logitsが存在する場合、推論結果を取得
if hasattr(outputs, "logits"):
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=-1).item()
    print(f" 推論結果: {predicted_class}")
else:
    print(" `logits`が出力に含まれていません")


モデル出力の内容: SequenceClassifierOutput(loss=None, logits=tensor([[0.2551, 0.0775]], device='cuda:0', dtype=torch.float16), hidden_states=None, attentions=None)
 推論結果: 0
